In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import matplotlib
# matplotlib.use("TkAgg")
%matplotlib tk
import matplotlib.pyplot as plt

## 2D dataset

In [3]:
num_points = 100
X1 = np.linspace(-2.5, 1.9, num_points)
# X1 = np.linspace(-2.5, 2.5, num_points)
# X2 = np.linspace(-2.5, 3, num_points)
X2 = np.linspace(-2.2, 2.1, num_points)
X1, X2 = np.meshgrid(X1, X2)
Y = np.sin(np.sqrt(X1**2 + X2**2))*2-1. - 0.1*(X1)+0.02*(X2)

####Scaling the data to range -1,1
X1 = 2*(X1 - X1.min())/(X1.max() - X1.min()) -1
X2 = 2*(X2 - X2.min())/(X2.max() - X2.min()) -1
Y = 2*(Y - Y.min())/(Y.max() - Y.min()) -1
Y = Y/2

x1 = X1.reshape(-1)
x2 = X2.reshape(-1)

xx = torch.Tensor(np.c_[x1, x2])
yy = torch.Tensor(Y.reshape(-1,1))


In [4]:
%matplotlib tk
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X1, X2, Y, cmap='plasma')
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('Y')
plt.show()

/home/tsuman/Program_Files/Python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
from convex_lib import ConvexNN

In [27]:
class ConvexNN_minima_v0(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, actf=nn.LeakyReLU):
        super().__init__()
        self.minima = nn.Parameter(torch.zeros(1, input_dim))
        self.layer0 = nn.Linear(input_dim, hidden_dim)
        self.actf0 = actf()
        self.layer1 = nn.Linear(hidden_dim, 1)
        
    def forward(self,x):
        h = x
        with torch.no_grad():
            self.layer1.weight.data.abs_()

#             print(torch.norm(self.layer0.weight.data, dim=1).shape, self.layer0.weight.data.shape)
#             l0norm = torch.norm(self.layer0.weight.data, dim=1, keepdim=True)
#             l0norm[l0norm<1] = 1.
#             print(torch.count_nonzero(l0norm>1))
#             self.layer0.weight.data /= l0norm
        
        h_lin = self.actf0(self.layer0(h))
        h_dist = torch.norm(x-self.minima, dim=1, keepdim=True)
#         print(h_dist.shape)
#         print(h_lin.shape)

        h = h_lin + h_dist
        
        h = self.layer1(h)
        
        return h

In [28]:
ConvexNN_minima_v0(2, 5)(xx)

tensor([[1.5994],
        [1.5847],
        [1.5702],
        ...,
        [0.9364],
        [0.9511],
        [0.9661]], grad_fn=<AddmmBackward>)

In [29]:
#### This not producing the
class ConvexNN_minima_v1(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, actf=nn.LeakyReLU):
        super().__init__()
        self.minima = nn.Parameter(torch.zeros(1, input_dim))
        self.layer0 = nn.Linear(input_dim, hidden_dim)
        self.actf0 = actf()
        self.layer1 = nn.Linear(hidden_dim, 1)
        
    def forward(self,x):
        h = x
        with torch.no_grad():
            self.layer1.weight.data.abs_()
#             self.layer1.weight.data /= self.layer1.weight.data.abs().sum()
            
#             l0norm = torch.norm(self.layer0.weight.data, dim=1, keepdim=True)
#             self.layer0.weight.data /= l0norm
        
        h_lin = self.layer1(self.actf0(self.layer0(h)))
        h_dist = torch.norm(x-self.minima, dim=1, keepdim=True)

        h = h_lin + h_dist
        
        return h

In [30]:
ConvexNN_minima_v1(2, 5)(xx)

tensor([[2.0184],
        [2.0002],
        [1.9822],
        ...,
        [2.1176],
        [2.1315],
        [2.1455]], grad_fn=<AddBackward0>)

In [31]:
# LVLs = np.linspace(sim.min(), sim.max(), 20)
LVLs = 20

cvxNet = ConvexNN_minima_v0(2, 5)
y_ = cvxNet(xx).data.cpu().numpy().reshape(Y.shape)

plt.figure(figsize=(8,8))
plt.contourf(X1, X2, y_, levels=LVLs)
cs = plt.contour(X1, X2, y_, levels=LVLs, linestyles="None", colors="k", linewidths=1)
plt.clabel(cs, cs.levels, inline=True, fontsize=10, fmt="%1.2f")
minima = xx[y_.argmin()]
plt.scatter(*minima.tolist(), s=100, edgecolors="red")

In [32]:
EPOCHS = 1000
actf = nn.LeakyReLU
# actf = nn.ELU
learning_rate = 0.01
criterion = nn.MSELoss()

# Convex NN

In [33]:
# cvxNet = ConvexNN([2, 10, 1], actf)
cvxNet = ConvexNN_minima_v0(2, 50, actf)
# cvxNet = ConvexNN_minima_v1(2, 10, actf)

optimizer = torch.optim.Adam(cvxNet.parameters(), lr=learning_rate)

In [42]:
%matplotlib tk
fig = plt.figure(figsize=(15,6))
ax = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122)

for epoch in range(EPOCHS):

    yout = -cvxNet(xx)    
    loss = criterion(yout, yy)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch%100 == 0 or epoch==EPOCHS-1:
        print(f'Epoch: {epoch}, Loss:{float(loss)}')
        ax.clear()
        ax.scatter(X1, X2, yy.data.numpy().reshape(-1), marker= '.')
        ax.scatter(X1, X2, yout.data.numpy().reshape(-1), color='r', marker='.')
        ax2.clear()
        ax2.contourf(X1, X2, yout.data.numpy().reshape(Y.shape), levels=20)

        fig.canvas.draw()
        plt.pause(0.01)
plt.close()

Epoch: 0, Loss:0.026791507378220558
Epoch: 100, Loss:0.026388827711343765
Epoch: 200, Loss:0.02519584819674492
Epoch: 300, Loss:0.024125004187226295
Epoch: 400, Loss:0.023888198658823967
Epoch: 500, Loss:0.02335393987596035
Epoch: 600, Loss:0.024673495441675186
Epoch: 700, Loss:0.022621963173151016
Epoch: 800, Loss:0.02321830578148365
Epoch: 900, Loss:0.022515254095196724
Epoch: 999, Loss:0.022972993552684784


In [35]:
cvxNet.layer0.weight.data.norm(dim=1)

tensor([0.3958, 0.3867, 0.4709, 0.1463, 0.4385, 0.4338, 0.4098, 0.5459, 0.2794,
        0.6009, 0.2871, 0.4855, 0.6090, 0.4902, 0.1086, 3.8824, 0.1455, 0.5257,
        0.0280, 0.2510, 0.1976, 0.4464, 0.1144, 0.3917, 0.3422, 0.3517, 0.2032,
        0.5200, 0.4894, 0.3625, 0.3769, 0.5733, 0.5405, 0.3550, 0.3466, 0.5240,
        0.2835, 0.6180, 0.4909, 0.4591, 0.3017, 0.3123, 0.3099, 0.0379, 0.5466,
        0.3884, 0.4474, 0.4902, 0.1275, 0.4245])

In [36]:
per_neuron_mag = cvxNet.layer0.weight.data.norm(dim=1) * cvxNet.layer1.weight.data.squeeze()
per_neuron_mag.sum()

tensor(0.5169)

In [37]:
cvxNet.layer1.weight.data

tensor([[-2.5402e-04, -3.3705e-04, -2.3886e-04, -2.5079e-04, -2.4785e-04,
         -2.3056e-04, -2.5291e-04, -2.2721e-04, -2.4713e-04, -2.4995e-04,
         -1.9773e-04, -1.6426e-04, -2.3448e-04, -2.1725e-04, -2.0446e-04,
          1.3419e-01, -2.4791e-04, -2.3788e-04, -2.4779e-04, -2.4915e-04,
         -1.9131e-04, -1.6071e-04, -2.5169e-04, -9.6318e-05, -1.0748e-04,
         -2.4548e-04, -8.1657e-05, -2.5043e-04, -2.2578e-04, -2.3718e-04,
         -2.4676e-04, -1.7683e-04, -2.4451e-04, -2.7423e-04, -2.0954e-04,
         -2.4791e-04, -1.7524e-04, -2.1794e-04, -2.4795e-04, -2.4513e-04,
         -2.4899e-04, -2.5484e-04, -2.4885e-04, -2.2049e-04, -1.8515e-04,
         -2.2579e-04, -2.1067e-04, -2.1444e-04, -2.2881e-04, -2.5484e-05]])

In [38]:
per_neuron_mag

tensor([-1.0055e-04, -1.3033e-04, -1.1248e-04, -3.6694e-05, -1.0869e-04,
        -1.0003e-04, -1.0364e-04, -1.2404e-04, -6.9038e-05, -1.5020e-04,
        -5.6763e-05, -7.9755e-05, -1.4280e-04, -1.0649e-04, -2.2199e-05,
         5.2096e-01, -3.6066e-05, -1.2505e-04, -6.9416e-06, -6.2526e-05,
        -3.7809e-05, -7.1748e-05, -2.8793e-05, -3.7728e-05, -3.6782e-05,
        -8.6329e-05, -1.6596e-05, -1.3023e-04, -1.1049e-04, -8.5987e-05,
        -9.2995e-05, -1.0138e-04, -1.3215e-04, -9.7357e-05, -7.2627e-05,
        -1.2991e-04, -4.9684e-05, -1.3469e-04, -1.2172e-04, -1.1253e-04,
        -7.5117e-05, -7.9592e-05, -7.7131e-05, -8.3496e-06, -1.0120e-04,
        -8.7699e-05, -9.4244e-05, -1.0511e-04, -2.9167e-05, -1.0819e-05])

In [39]:
per_neuron_mag.abs() > 1

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False])

In [40]:
%matplotlib tk

y_ = yout.data.cpu().numpy().reshape(Y.shape)

fig = plt.figure(figsize=(8,6))
ax = fig.gca(projection='3d')
ax.view_init(49, -71)
ax.plot_surface(X1, X2, y_, cmap='plasma', alpha=0.9)
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('Y')
plt.pause(1)
plt.show()

/home/tsuman/Program_Files/Python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  


## Plotting Contour Plot

In [43]:
# LVLs = np.linspace(sim.min(), sim.max(), 20)
LVLs = 20

y_ = cvxNet(xx).data.cpu().numpy().reshape(Y.shape)

plt.figure(figsize=(8,8))
plt.contourf(X1, X2, y_, levels=LVLs)
cs = plt.contour(X1, X2, y_, levels=LVLs, linestyles="None", colors="k", linewidths=1)
plt.clabel(cs, cs.levels, inline=True, fontsize=10, fmt="%1.2f")
minima = xx[y_.argmin()]
plt.scatter(*minima.tolist(), s=100, edgecolors="red")

In [21]:
min_val = y_.min()
min_val

-0.37873057

In [22]:
minima = xx[None, y_.argmin()]
minima

tensor([[-0.4949,  0.1919]])